In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import linregress

In [ ]:
file_name = "../physical_circles.h5"
store = pd.HDFStore(file_name, mode="r")
test_data = store.get("/circles/true")
test_data.index.name = "time"

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.scatter(test_data.position.x, test_data.position.y, test_data.index)

In [ ]:
tables={}
names = ["/circles/true"]
timestep_bases = [1, 2, 5]
# timestep_magnitudes = [0.001, 0.01, 0.1, 1, 10]
timestep_magnitudes = [0.01, 0.1, 1, 10]
timesteps = []
for magnitude in timestep_magnitudes:
    for base in timestep_bases:
        timesteps.append(base*magnitude)

for timestep in timesteps:
    names.append(f"/circles/ts_{int(1000*timestep)}")
for name in names:
    tables[name] = store.get(name)
    print(f"{name}: \n{tables[name].head()}")

In [ ]:
# for name in names:
#     table = tables[name]
#     table["rad"] = table.apply(lambda row: np.hypot(row.position.x, row.position.y), axis=1)

In [ ]:
tables["ts_200"].tail()

In [ ]:
fig, ax = plt.subplots()
# print(names)
# print(tables)
for name in names:
    if name.startswith("true"): continue
    # print(name)
    table = tables[name]
    # generate radial difference:
    table["rad_diff"] = table.apply(lambda row: row.rad - 1e6, axis=1)
    # table.head()
    ax.plot(table["rad_diff"])

In [ ]:
fig, ax = plt.subplots()
for name in names:
    if name.startswith("true"): continue
    # print(name)
    table = tables[name]
    # generate radial difference:
    table["rad_diff_pow"] = table.apply(lambda row: np.power(row.rad_diff, 1/3), axis=1)
    # table.head()
    ax.plot(table.index, table["rad_diff_pow"])

In [ ]:
_table = tables["ts_20000"]
regression = linregress(_table.index, _table["rad_diff_pow"], alternative="greater")
print(regression)

In [ ]:
slopes = []
for name in names:
    if name.startswith("true"): continue
    table = tables[name]
    regression = linregress(table.index, table["rad_diff_pow"])
    # print(table.tail())
    print(regression)
    slopes.append(regression.slope)
# plt.scatter(timesteps, slopes)
print(slopes)


In [ ]:
# looks maybe logarithmic, or maybe 1/x^2 or something.
# plt.scatter(np.log10(timesteps))
plt.scatter(np.log10(timesteps), np.log10(slopes))

In [ ]:
# not perfectly linear in logxlog space, but probably close enough to get a result
reg = linregress(np.log10(timesteps), np.log10(slopes))
# print(reg.slope, reg.intercept)
# log10(slopes) = reg.slope*log10(timestep) + reg.intercept
# slope is (average) increase in radial error over time
# to find a timestep for a threshold value of error, invert the regression
inv_reg = linregress(np.log10(slopes), np.log10(timesteps))


In [ ]:
# then use the values to solve for a timestamp:
threshold_precision = 4e-6 # using 4 since base radial value is 4, and we want 1 in 10^6 precision, say
threshold_timestep_l10 = inv_reg.slope*np.log10(threshold_precision) + inv_reg.intercept
threshold_timestep = np.power(10, threshold_timestep_l10)
print(threshold_timestep)
# 1 microsecond gets us approximately ppm position resolution.
# since it's log scale, 10 µs -> 10 ppm, 0.1 ms -> 100 ppm etc. (roughly)